Ses Kayıtlarını 1,5 Saniyelik Chunklara Ayırma

In [2]:
import os
from pydub import AudioSegment

# WAV dosyalarının bulunduğu dizin
wav_dir = "ses/dosyalarının/yolu"
chunk_dir = "C:/.../chunks"
os.makedirs(chunk_dir, exist_ok=True)  # Chunk'lar için çıkış dizini oluştur

# WAV dosyalarının listesi
wav_files = os.listdir(wav_dir)

# Her WAV dosyasını işleyin
for wav_file in wav_files:
    if wav_file.endswith('.wav'):
        # WAV dosyasını yükle
        audio = AudioSegment.from_wav(os.path.join(wav_dir, wav_file))
        # 1.5 saniyelik parçalara böl (milisaniye cinsinden 1500ms)
        chunk_length_ms = 1500  # 1.5 saniye
        chunks = [audio[i:i + chunk_length_ms] for i in range(0, len(audio), chunk_length_ms)]
        
        # Parçaları kaydet
        for i, chunk in enumerate(chunks):
            chunk_name = f"{os.path.splitext(wav_file)[0]}_chunk_{i}.wav"
            chunk.export(os.path.join(chunk_dir, chunk_name), format="wav")

print("Tüm dosyalar başarıyla 1.5 saniyelik parçalara bölündü.")

Tüm dosyalar başarıyla 1.5 saniyelik parçalara bölündü.


Seslerin MFCC Feature'larını Çıkarma

In [ ]:
import os
import librosa
import numpy as np

# Ses dosyalarının bulunduğu dizin
chunk_dir = "C:/.../chunks"

# MFCC özelliklerinin kaydedileceği dizin
mfcc_dir = 'C:/.../mfcc_chunks'
os.makedirs(mfcc_dir, exist_ok=True)

# MFCC parametreleri
n_mfcc = 128
frame_length = 25  # milisaniye
frame_stride = 10  # milisaniye
max_length = 128

# Dosyaları işleme
for chunk_file in os.listdir(chunk_dir):
    if chunk_file.endswith('.wav'):
        chunk_path = os.path.join(chunk_dir, chunk_file)
        print(f"İşleniyor: {chunk_file}")
        try:
            # Ses dosyasını yükleme
            y, sr = librosa.load(chunk_path, sr=None)
            hop_length = int(frame_stride * sr / 1000)
            n_fft = int(frame_length * sr / 1000)

            # MFCC çıkarma
            mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc, hop_length=hop_length, n_fft=n_fft)

            # Sabit uzunlukta MFCC oluşturma
            if mfcc.shape[1] > max_length:
                mfcc = mfcc[:, :max_length]
            elif mfcc.shape[1] < max_length:
                pad_width = max_length - mfcc.shape[1]
                mfcc = np.pad(mfcc, ((0, 0), (0, pad_width)), mode='constant')

            # MFCC'yi düzleştirme
            mfcc_flat = mfcc.flatten()

            # MFCC dosyasını kaydet
            mfcc_file = os.path.join(mfcc_dir, chunk_file.replace('.wav', '.npy'))
            np.save(mfcc_file, mfcc_flat)
        except Exception as e:
            print(f"{chunk_file} işlenirken hata oluştu: {e}")


Model Eğitimi ve Model Doğruluğunu Görme

In [ ]:
import os
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
from collections import Counter
import joblib

# MFCC dosyalarının bulunduğu dizin
# MFCC dosyalarının bulunduğu dizin
mfcc_dir = 'C:/.../mfcc_chunks'

# Veriyi yükleme
X, y = [], []
for mfcc_file in os.listdir(mfcc_dir):
    if mfcc_file.endswith('.npy'):
        try:
            mfcc_path = os.path.join(mfcc_dir, mfcc_file)
            mfcc = np.load(mfcc_path)
            X.append(mfcc.flatten())  # Düzleştirme işlemi burada yapılıyor
            label = mfcc_file.split('_')[0]
            y.append(label)
        except Exception as e:
            print(f"{mfcc_file} yüklenirken hata oluştu: {e}")

X = np.array(X)
y = np.array(y)

# Etiketleri sayısal değerlere dönüştürme
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Sınıf ağırlıklarını hesaplama
class_weights = compute_class_weight('balanced', classes=np.unique(y), y=y)
sample_weights = np.array([class_weights[class_label] for class_label in y])

# Sınıf dağılımını kontrol et
print("Sınıf dağılımı:", Counter(y))

# Normalizasyon
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Veri kümesini eğitim ve test kümelerine ayırma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

# MLP Modeli
model = MLPClassifier(hidden_layer_sizes=(128, 64), activation='relu', solver='adam', max_iter=1000, random_state=42)
model.fit(X_train, y_train)

# Modelin doğruluğunu değerlendirme
accuracy = model.score(X_test, y_test)
print(f"Model Doğruluğu: {accuracy:.2f}")

# Sınıflandırma Raporu
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

# Karışıklık Matrisi
conf_matrix = confusion_matrix(y_test, y_pred)
print("Karışıklık Matrisi:")
print(conf_matrix)

# Çapraz doğrulama
scores = cross_val_score(model, X, y, cv=5)
print(f"Çapraz Doğrulama Sonuçları: {scores}")
print(f"Ortalama Doğruluk: {scores.mean():.2f}")

# Modeli kaydetme
joblib.dump(model, 'C:/.../mlp_model.pkl')
joblib.dump(label_encoder, 'C:/.../label_encoder.pkl')
joblib.dump(scaler, 'C:/.../scaler.pkl')



Test Doğruluğunu Görme

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

def evaluate_model(model, X_test, y_test, label_encoder):
    # Test veri seti üzerinde doğruluk değerlendirmesi
    accuracy = model.score(X_test, y_test)
    print(f"Test Doğruluğu (Accuracy): {accuracy:.4f}")
    
    # Sınıflandırma Raporu ve Karışıklık Matrisi
    y_pred = model.predict(X_test)
    print("Sınıflandırma Raporu:")
    if label_encoder:
        target_names = label_encoder.classes_ if hasattr(label_encoder, 'classes_') else None
        print(classification_report(y_test, y_pred, target_names=target_names))
    else:
        print(classification_report(y_test, y_pred))
    
    print("Karışıklık Matrisi:")
    print(confusion_matrix(y_test, y_pred))

# Örnek kullanım
evaluate_model(model, X_test, y_test, label_encoder)

Tahmin Yaptırma

In [ ]:
import numpy as np
import sounddevice as sd
import joblib
import librosa
import soundfile as sf
import time

# Eğitilen model, scaler ve label encoder yolları
model_kayit_yolu = 'C:/...mlp_model.pkl'
label_encoder_yolu = 'C:/.../label_encoder.pkl'
scaler_yolu = 'C:/.../scaler.pkl'

# Model, LabelEncoder ve Scaler'ı yükle
model = joblib.load(model_kayit_yolu)
label_encoder = joblib.load(label_encoder_yolu)
scaler = joblib.load(scaler_yolu)

# Sınıf isimlerini LabelEncoder'dan al
sinif_isimleri = label_encoder.classes_
print("Sınıflar:", sinif_isimleri)

# Mikrofon parametreleri
saniye_basina_ornek = 44100
saniye = 5
kanal_sayisi = 1

try:
    while True:
        print("Konuşun...")
        # Ses kaydı
        ses = sd.rec(int(saniye_basina_ornek * saniye), samplerate=saniye_basina_ornek, channels=kanal_sayisi, dtype='float32')
        sd.wait()
        
        # Dinamik dosya ismi
        kayit_yolu = f"kayit_{int(time.time())}.wav"
        sf.write(kayit_yolu, np.squeeze(ses), saniye_basina_ornek)
        
        # MFCC çıkarma
        y, sr = librosa.load(kayit_yolu, sr=saniye_basina_ornek)
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=128, hop_length=512, n_fft=2048)
        mfcc = np.mean(mfcc.T, axis=1)

        # Normalizasyon (Scaler ile)
        mfcc_normalized = scaler.transform(mfcc.reshape(1, -1))

        # Model tahmini
        tahmin_indeksi = model.predict(mfcc_normalized)[0]
        tahmin_isim = sinif_isimleri[tahmin_indeksi]

        print("Tahmin edilen kişi:", tahmin_isim)

except KeyboardInterrupt:
    print("\nProgram durduruldu. Çıkış yapılıyor...")
